In [1]:
import pyspark

In [2]:
pyspark.__file__

'/usr/local/spark/python/pyspark/__init__.py'

In [3]:
from pyspark.sql import SparkSession

In [11]:
spark = SparkSession.builder.master("local[*]").appName('test').getOrCreate()

In [12]:
spark1 = SparkSession.builder \
    .master("spark://spark:7077") \
    .appName("Jupyter PySpark") \
    .getOrCreate()


#### Question1
Install Spark and PySpark

Install Spark
Run PySpark
Create a local spark session
Execute spark.version.

What's the output?



In [165]:

spark.version

'3.5.0'

In [22]:
df = spark.read.csv('fhv_tripdata_2019-10.csv', header=True)

In [23]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   NULL|                B00014|
|     B00021         |2019-10-01 00:00:4

In [24]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', StringType(), True), StructField('DOlocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [30]:
from pyspark.sql import types

In [34]:
schema = types.StructType(
    [types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropOff_datetime', types.TimestampType(), True), 
    types.StructField('PUlocationID', types.IntegerType(), True), 
    types.StructField('DOlocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True), 
    types.StructField('Affiliated_base_number', types.StringType(), True)]
    
    )


In [35]:
df = spark.read.schema(schema).csv('fhv_tripdata_2019-10.csv', header=True)

In [36]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   NULL|                B00014|
|     B00021         |2019-10-01 00:00:4

In [37]:
df.head()

Row(dispatching_base_num='B00009', pickup_datetime=datetime.datetime(2019, 10, 1, 0, 23), dropOff_datetime=datetime.datetime(2019, 10, 1, 0, 35), PUlocationID=264, DOlocationID=264, SR_Flag=None, Affiliated_base_number='B00009')

In [68]:
df= df.repartition(6)

In [60]:
df.write.parquet('partitions/2019_01')

In [43]:
num_partitions = df.rdd.getNumPartitions()
print("Number of partitions:", num_partitions)


Number of partitions: 6


In [80]:
from pyspark.sql import functions as F
df = spark.read.parquet('partitions/2019_01/*')
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02688|2019-10-02 10:59:00|2019-10-02 11:34:46|         264|         107|   NULL|                B02688|
|              B01843|2019-10-02 18:24:00|2019-10-02 22:21:00|         264|         264|   NULL|                  NULL|
|              B02293|2019-10-01 11:05:20|2019-10-01 12:49:00|         107|         168|   NULL|                B02293|
|              B03162|2019-10-03 07:16:31|2019-10-03 08:50:47|         210|         236|   NULL|                B03162|
|              B01190|2019-10-02 08:41:46|2019-10-02 08:51:41|         264|          49|   NULL|                B01190|
|              B00254|2019-10-02 13:20:4

##### Question 2:
FHV October 2019

Read the October 2019 FHV into a Spark Dataframe with a schema as we did in the lessons.

Repartition the Dataframe to 6 partitions and save it to parquet.

What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

1MB
6MB
25MB
87MB


#### Answer 

each file is 5.9 MB which approximately to :
###### 6MB

#### Question 3:
Count records

How many taxi trips were there on the 15th of October?

Consider only trips that started on the 15th of October.

108,164
12,856
452,470
62,610


#### Answer
##### 62610


In [78]:
df.registerTempTable('fhvhv')

/usr/local/spark/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [84]:
spark.sql(""" select count(*) from fhvhv where DATE(pickup_datetime) = '2019-10-15'; """).show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



#### Question 4:
Longest trip for each day

What is the length of the longest trip in the dataset in hours?

631,152.50 Hours
243.44 Hours
7.68 Hours
3.32 Hours


#### Answer 
631,152.50 Hours

In [129]:
from pyspark.sql.functions import col, asc,desc


In [131]:
df \
    .withColumn('duration_seconds', df.dropOff_datetime.cast('long')-df.pickup_datetime.cast('long')) \
    .orderBy(col('duration_seconds').desc()) \
    .show()


+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|duration_seconds|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------------+
|              B02832|2019-10-28 09:00:00|2091-10-28 09:30:00|         264|         264|   NULL|                B02832|      2272149000|
|              B02832|2019-10-11 18:00:00|2091-10-11 18:30:00|         264|         264|   NULL|                B02832|      2272149000|
|              B02416|2019-10-31 23:46:33|2029-11-01 00:13:00|        NULL|        NULL|   NULL|                B02416|       315620787|
|     B00746         |2019-10-01 21:43:42|2027-10-01 21:45:23|         159|         264|   NULL|       B00746         |       252460901|
|              B02921|2019-10-17 14:00:00

In [136]:
spark.sql("""
SELECT
    pickup_datetime, dropoff_datetime,
    (unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime)) / 3600  AS duration
FROM
    fhvhv
ORDER BY
    duration DESC
""").show()


+-------------------+-------------------+------------------+
|    pickup_datetime|   dropoff_datetime|          duration|
+-------------------+-------------------+------------------+
|2019-10-28 09:00:00|2091-10-28 09:30:00|          631152.5|
|2019-10-11 18:00:00|2091-10-11 18:30:00|          631152.5|
|2019-10-31 23:46:33|2029-11-01 00:13:00| 87672.44083333333|
|2019-10-01 21:43:42|2027-10-01 21:45:23| 70128.02805555555|
|2019-10-17 14:00:00|2020-10-18 00:00:00|            8794.0|
|2019-10-26 21:26:00|2020-10-26 21:36:00| 8784.166666666666|
|2019-10-30 12:30:04|2019-12-30 13:02:08|1464.5344444444445|
|2019-10-25 07:04:57|2019-12-08 07:54:33|1056.8266666666666|
|2019-10-25 07:04:57|2019-12-08 07:21:11|1056.2705555555556|
|2019-10-01 13:47:17|2019-11-03 15:20:28| 793.5530555555556|
|2019-10-01 07:21:12|2019-11-03 08:44:21| 793.3858333333334|
|2019-10-01 13:41:00|2019-11-03 14:58:51|          793.2975|
|2019-10-01 18:43:20|2019-11-03 19:43:13| 792.9980555555555|
|2019-10-01 18:43:46|201

#### Question 5:
User Interface

Spark’s User Interface which shows the application's dashboard runs on which local port?

80
443
4040
8080


#### Answer 
4040


#### Question 6:
Least frequent pickup location zone

Load the zone lookup data into a temp view in Spark
Zone Data

Using the zone lookup data and the FHV October 2019 data, what is the name of the LEAST frequent pickup location Zone?

East Chelsea
Jamaica Bay
Union Sq
Crown Heights North


#### Answer 

##### Jamaica Bay


In [138]:
df_zones = spark.read.csv('taxi_zone_lookup.csv/', header=True)


In [141]:
df_zones.write.parquet('zones')


In [142]:
df_zone = spark.read.parquet('zones/')


In [143]:
df_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [150]:
df_join=df.join(df_zone, df.PUlocationID == df_zone.LocationID)

In [154]:
df_join.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+---------+--------------+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|LocationID|  Borough|          Zone|service_zone|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+---------+--------------+------------+
|              B02688|2019-10-02 10:59:00|2019-10-02 11:34:46|         264|         107|   NULL|                B02688|       264|  Unknown|            NV|         N/A|
|              B01843|2019-10-02 18:24:00|2019-10-02 22:21:00|         264|         264|   NULL|                  NULL|       264|  Unknown|            NV|         N/A|
|              B02293|2019-10-01 11:05:20|2019-10-01 12:49:00|         107|         168|   NULL|                B02293|       107|Manhattan|      Gramercy|

In [159]:
df_join = df_join.drop('LocationID')

In [162]:
df_join.show(2)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-------+----+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|Borough|Zone|service_zone|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-------+----+------------+
|              B02688|2019-10-02 10:59:00|2019-10-02 11:34:46|         264|         107|   NULL|                B02688|Unknown|  NV|         N/A|
|              B01843|2019-10-02 18:24:00|2019-10-02 22:21:00|         264|         264|   NULL|                  NULL|Unknown|  NV|         N/A|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-------+----+------------+
only showing top 2 rows



In [170]:
pickup_counts = df_join.groupBy("Zone").count()
print(pickup_counts.show())
# Find the least frequent pickup location zone
least_frequent_zone = pickup_counts.orderBy("count").first()["Zone"]
print(least_frequent_zone)

+--------------------+-----+
|                Zone|count|
+--------------------+-----+
|           Homecrest| 1295|
|              Corona| 7175|
|    Bensonhurst West| 1880|
|         Westerleigh| 1317|
|Charleston/Totten...| 2533|
|      Newark Airport| 2532|
|          Douglaston|  291|
|East Concourse/Co...| 3294|
|          Mount Hope| 3973|
|      Pelham Parkway| 1522|
|         Marble Hill|  116|
|           Rego Park| 1697|
|Upper East Side S...| 2358|
|Heartland Village...| 1054|
|       Dyker Heights|  690|
|   Kew Gardens Hills|  906|
|     Jackson Heights|10952|
|             Bayside| 2547|
|TriBeCa/Civic Center| 1918|
|      Yorkville West|  840|
+--------------------+-----+
only showing top 20 rows

None
Jamaica Bay
